# Creating and Valuing a CDS Index

Example CDS Index Valuation

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

####################################################################
# FINANCEPY BETA Version 0.200 - This build:  14 Jul 2021 at 13:18 #
# **** NEW PEP8 COMPLIANT VERSION -- PLEASE UPDATE YOUR CODE  **** #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



In [3]:
valuation_date = Date(7, 2, 2006)
settlement_date = valuation_date.add_weekdays(0)
print(settlement_date)

07-FEB-2006


## Build Ibor Curve

Build the Ibor curve to value cash flows

In [4]:
depos = []
dcType = DayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FrequencyTypes.SEMI_ANNUAL
swapType = SwapTypes.PAY
swap1 = IborSwap(settlement_date,"1Y",swapType,0.0502,fixedFreq,dcType)
swap2 = IborSwap(settlement_date,"2Y",swapType,0.0502,fixedFreq,dcType)
swap3 = IborSwap(settlement_date,"3Y",swapType,0.0501,fixedFreq,dcType)
swap4 = IborSwap(settlement_date,"4Y",swapType,0.0502,fixedFreq,dcType)
swap5 = IborSwap(settlement_date,"5Y",swapType,0.0501,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5]

libor_curve = IborSingleCurve(valuation_date, depos, [], swaps)

## Build Index Curve

We treat an index as a CDS contract with a flat CDS curve at the CDS index spread for the same maturity

In [5]:
step_in_date = valuation_date

In [7]:
index_coupon = 0.0048375
maturity_date = Date(20, 6, 2010)

cdsMarketContracts = []
cds = CDS(valuation_date, maturity_date, index_coupon)
cdsMarketContracts.append(cds)

recovery_rate = 0.40
index_curve = CDSCurve(valuation_date, cdsMarketContracts,libor_curve,recovery_rate)

## Create CDS Index and Perform Valuation

The CDS Index is treated as a CDS with the coupon equal to the CDS Index coupon

In [8]:
cdsRecovery = 0.40
notional = 10.0 * ONE_MILLION
long_protection = True
index_coupon = 0.004 

cdsIndexContract = CDS(step_in_date, maturity_date, index_coupon, notional, long_protection)

In [9]:
spd = cdsIndexContract.par_spread(valuation_date, index_curve, cdsRecovery) * 10000
print("PAR SPREAD", spd)

PAR SPREAD 48.37497288767587


In [10]:
v = cdsIndexContract.value(valuation_date, index_curve, cdsRecovery)
print("DIRTY VALUE", v['dirty_pv'])
print("CLEAN VALUE", v['clean_pv'])

FULL VALUE 27151.269866109884
CLEAN VALUE 32595.714310554293


In [11]:
p = cdsIndexContract.clean_price(valuation_date, index_curve, cdsRecovery)
print("CLEAN PRICE", p)

CLEAN PRICE 99.67404290334386


In [12]:
accrued_days = cdsIndexContract.accrued_days()
print("ACCRUED DAYS", accrued_days)

accrued_interest = cdsIndexContract.accrued_interest()
print("ACCRUED COUPON", accrued_interest)

ACCRUED DAYS 49.0
ACCRUED COUPON -5444.444444444444


In [13]:
prot_pv = cdsIndexContract.protection_leg_pv(valuation_date, index_curve, cdsRecovery)
print("PROTECTION LEG PV", prot_pv)

PROTECTION LEG PV 188277.24186998222


In [14]:
premPV = cdsIndexContract.premium_leg_pv(valuation_date, index_curve, cdsRecovery)
print("PREMIUM LEG PV", premPV)

PREMIUM LEG PV 161125.97200387233


In [15]:
rpv01 = cdsIndexContract.risky_pv01(valuation_date, index_curve)
print("DIRTY RPV01", rpv01['dirty_rpv01'])
print("CLEAN RPV01", rpv01['clean_rpv01'])

DIRTY RPV01 4.028149300096809
CLEAN RPV01 3.8920381889856976


Copyright (c) 2020 Dominic O'Kane